In [1]:
import env
import pandas as pd
import numpy as np
import utilities as utils

from wrangle import wrangle_articles

In [2]:
all_articles_df, only_articles_with_text_df = wrangle_articles()

NaN values
Empty DataFrame
Columns: [rows, columns]
Index: []
--------------------------------
Empty values
      rows  columns
0      120        1
1      122        1
2      152        1
3      165        1
4      172        1
..     ...      ...
627  24267        1
628  25130        1
629  25152        1
630  25701        1
631  25746        1

[632 rows x 2 columns]
--------------------------------


In [3]:
all_articles_df.to_csv("all_articles.csv")
only_articles_with_text_df.to_csv("only_articles_with_text.csv")